In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,make_pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Bidirectional
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.layers import Flatten
from sklearn.model_selection import train_test_split

In [ ]:
CFG = [
    '/kaggle/input/software-effort-estimation-datasets/albrecht.csv',
    '/kaggle/input/software-effort-estimation-datasets/china.csv',
    '/kaggle/input/software-effort-estimation-datasets/desharnais.csv',
    '/kaggle/input/software-effort-estimation-datasets/finnish.csv',
    '/kaggle/input/software-effort-estimation-datasets/isbsg10.csv',
    '/kaggle/input/software-effort-estimation-datasets/kemerer.csv',
    '/kaggle/input/software-effort-estimation-datasets/kitchenham.csv',
    '/kaggle/input/software-effort-estimation-datasets/maxwell.csv',
    '/kaggle/input/software-effort-estimation-datasets/miyazaki94.csv'
]

In [3]:
cocomo_path= '/kaggle/input/cocomo/5.cocomo81.csv'

In [4]:
def load_cocomo_data():
    data = pd.read_csv(cocomo_path)
    X = data.iloc[:, :-1].values  # Features
    y = data.iloc[:, -1].values  # Effort
    return X, y

X, y = load_cocomo_data()

# Data Preprocessing

In [ ]:
def data_albrecht():
    df = pd.read_csv(CFG[0])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = MinMaxScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df


def data_china():
    
    df = pd.read_csv(CFG[1])
    df = df.drop(columns=['id','ID'])
    df_for_training = df.astype(float)
#     print(df.columns)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    
    trainX = []
    trainY = []
    n_future = 2   # Number of days we want to look into the future based on the past days.
    n_past = 2 # Number of past days we want to use to predict the future.

    for i in range(n_past, len(df_for_training_scaled) - n_future +1):
        trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
        trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, df_for_training.shape[1] - 1])

    trainX, trainY = np.array(trainX), np.array(trainY)

    return trainX, trainY


def data_desharnais():
    df = pd.read_csv(CFG[2])
    df = df.drop(columns=['Project','YearEnd', 'Language'])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df


def data_finnish():
    df = pd.read_csv(CFG[3])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = MinMaxScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df


def data_isbsg10():

    df = pd.read_csv(CFG[4])
    df = df.drop(columns = ['id', 'ID', 'Data_Quality', 'Year','AG', 'N_effort_level1',
           'N_PDR1', 'SDR', 'PET', 'PIT', 'I_Date', 'PAS',
           'Recording_Method', 'Resource_Level', 'MTS', 'ATS', 'R_PWE_NPA',
           'P_UWE', 'CASE_Tool', 'UM', 'HMA', 'Hardware1',
           'IDE', 'DT1', 'DBS1', 'CS1', 'WS1', 'MS1', 'OP1', 'RTA',
           'SP_CMMI', 'SP_ISO', 'SP_TICKIT', 'MIN_Defects', 'MAJ_Defects',
           'X_Defects', 'TOT_Defects', 'UB_BU', 'UB_L', 'UB_DU', 'UB_CU',
           'IMarket', 'T_Platform', 'D_Embedded', 'SE', 'SEA', 'SEM', 'E_Estimate',
           'E_Estimate_Method', 'DDE', 'DDEM', 'C_Estimate', 'CEC', 'CEM',
           'E_Tool', 'E_Comments', 'EC_Date', 'SR?', 'SR', 'R_FPC', 'R_FPA',
           'P_Defects', 'D_Defects', 'MIN_B_Defects', 'MAJ_B_Defects',
           'X_B_Defects', 'TOT_B_Defects', 'MIN_T_Defects', 'MAJ_T_Defects',
           'X_T_Defects', 'TOT_T_Defects','S_Defects', 'MIN_I_Defects', 'MAJ_I_Defects',
           'X_I_Defects', 'TOT_I_Defects'], axis = 1)
    # Replace "?" with numpy.nan
    df.replace('?', np.nan, inplace=True)
#     print(df.columns)
    
    # imputation transformer
    trf1 = ColumnTransformer([
        ('impute',SimpleImputer(strategy='constant', fill_value='Missing'),[14,15,16,17,18])
    ],remainder='passthrough')
    
    trf2 = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse_output=False,drop= 'first'),[0,1,2,3,4,5,6,7,8,9,10,11,12,13,15])
    ],remainder='passthrough')
    
    pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ])
    
    pipe.fit_transform(df)
    
    return pipe.fit_transform(df)


def data_kemerer():
    df = pd.read_csv(CFG[5])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df
    

    
def data_kitchenham():
    df = pd.read_csv(CFG[6])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = MinMaxScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df


def data_maxwell():
    df = pd.read_csv(CFG[7])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df


def data_miyazaki():
    df = pd.read_csv(CFG[8])
    df_for_training = df.astype(float)
    #LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    return df


In [ ]:
import pywt

In [5]:
def morlet_wavelet(t, f0=1.0, sigma=1.0):
    """
    Morlet wavelet function.
    t: time variable
    f0: center frequency
    sigma: bandwidth parameter
    """
    return np.exp(-0.5 * (t / sigma) ** 2) * np.cos(2 * np.pi * f0 * t)

In [7]:
from tensorflow.keras.layers import Layer

In [8]:
class MorletWaveletLayer(Layer):
    def __init__(self, units, w=5.0, **kwargs):
        super(MorletWaveletLayer, self).__init__(**kwargs)
        self.units = units
        self.w = w

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[-1], self.units),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs):
        # Reshape inputs to 2D for the dense operation
        input_reshaped = tf.reshape(inputs, [-1, inputs.shape[-1]])
        z = tf.matmul(input_reshaped, self.kernel)

        # Implement Morlet wavelet using TensorFlow operations
        wavelet_output = tf.cos(self.w * z) * tf.exp(-z**2 / 2)
        
        # Reshape back to 3D
        final= tf.reshape(wavelet_output, tf.concat([tf.shape(inputs)[:-1], [self.units]], axis=0))
        print(final.shape)
        return final

In [ ]:
class MorletWaveletLayer(Layer):
    def __init__(self, filters, f0=1.0, sigma=1.0, **kwargs):
        super(MorletWaveletLayer, self).__init__(**kwargs)
        self.filters = filters
        self.f0 = f0
        self.sigma = sigma

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(self.filters, input_shape[-1]),
            initializer='glorot_uniform',
            trainable=True,
        )
        self.built = True

    def call(self, inputs):
        time = np.linspace(-1, 1, inputs.shape[1])
        wavelet = morlet_wavelet(time, self.f0, self.sigma)
        wavelet = tf.convert_to_tensor(wavelet, dtype=tf.float32)
        wavelet = tf.reshape(wavelet, (1, -1, 1))
        
        wavelet_kernels = wavelet * tf.expand_dims(self.kernel, axis=1)
        
        # Ensure the wavelet kernels are properly shaped for convolution
        wavelet_kernels = tf.transpose(wavelet_kernels, [1, 2, 0])
        
        conv = tf.nn.conv1d(inputs, wavelet_kernels, stride=1, padding='SAME')
        return conv


In [9]:
totalX, totalY = load_cocomo_data()

In [ ]:
import numpy as np
############# NOT USING FOR NOW#######
class FireflyAlgorithm:
    def __init__(self, n_fireflies, n_iterations, alpha=0.5, beta=1.0, gamma=1.0):
        self.n_fireflies = n_fireflies
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

    def initialize_fireflies(self, bounds):
        fireflies = np.random.rand(self.n_fireflies, len(bounds))
        for i in range(len(bounds)):
            fireflies[:, i] = bounds[i][0] + fireflies[:, i] * (bounds[i][1] - bounds[i][0])
        return fireflies

    def calculate_light_intensity(self, firefly, objective_function):
        return objective_function(firefly)

    def move_firefly(self, firefly_i, firefly_j, beta):
        r = np.linalg.norm(firefly_i - firefly_j)
        return firefly_i + beta * np.exp(-self.gamma * r ** 2) * (firefly_j - firefly_i) + self.alpha * (np.random.rand(len(firefly_i)) - 0.5)

    def optimize(self, objective_function, bounds):
        fireflies = self.initialize_fireflies(bounds)
        light_intensities = np.array([self.calculate_light_intensity(firefly, objective_function) for firefly in fireflies])

        for iteration in range(self.n_iterations):
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if light_intensities[i] > light_intensities[j]:
                        fireflies[i] = self.move_firefly(fireflies[i], fireflies[j], self.beta)
                        light_intensities[i] = self.calculate_light_intensity(fireflies[i], objective_function)
        
        best_firefly = fireflies[np.argmin(light_intensities)]
        best_intensity = np.min(light_intensities)

        return best_firefly, best_intensity


In [10]:
trainX, testX, trainY, testY = train_test_split(totalX, totalY, test_size=0.25, random_state=42)

# Define the objective function
def objective_function(params):
    f0, sigma = params[0], params[1]
    
    # Create the Wavelet Neural Network
    input_shape = trainX.shape[1:]  # Assuming trainX.shape is (496, 2, 18), input_shape should be (2, 18)
    wnn = Sequential([
        BatchNormalization(input_shape=input_shape),
        Dense(64, activation='relu'),
        Dropout(0.1),
        Dense(32, activation='relu'),
        MorletWaveletLayer(filters=16, f0=1.0, sigma=1.0, input_shape=(None, 1)),  # Replace with appropriate units for your use case
        Flatten(),
        Dense(1)  # Adjust output dim to match trainY.shape[1]
    ])

    # Compile the model
    wnn.compile(optimizer='adam', loss='mse')
    #wnn.summary()
    wnn.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
    
    loss = wnn.evaluate(testX, testY, verbose=0)
    return loss


In [11]:
    # Create the Wavelet Neural Network
    input_shape = trainX.shape[1]  # Assuming trainX.shape is (496, 2, 18), input_shape should be (2, 18)
    wnn = Sequential([
        BatchNormalization(input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dropout(0.1),
        Dense(32, activation='relu'),
        MorletWaveletLayer(10),  # Replace with appropriate units for your use case
        Dense(1)  # Adjust output dim to match trainY.shape[1]
    ])

(None, 10)


In [12]:
wnn.compile(optimizer='adam', loss='mse')

In [13]:
wnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 morlet_wavelet_layer (Morle  (None, 10)               320       
 tWaveletLayer)                                                  
                                                                 
 dense_2 (Dense)             (None, 1)                 1

In [14]:
history = wnn.fit(trainX, trainY, epochs=100, batch_size=32, verbose=0)
# Evaluate model
predictions = wnn.predict(testX)

(None, 10)
(None, 10)
(None, 10)
1/1 [==============================] - 0s 120ms/step


In [15]:
mse = mean_squared_error(testY, predictions)

In [16]:
mse

338008.20830537344

In [17]:
def mean_magnitude_relative_error(y_true, y_pred):
    # Calculate relative error
    relative_error = np.abs((y_true - y_pred) / y_true)
    # Calculate mean magnitude of relative error (MMRE)
    mmre = np.mean(relative_error)
    
    return mmre

In [18]:
mean_magnitude_relative_error(testY,predictions)

0.9909676698965766